# Driver Drowsiness Detecting System
This model detects prolonged eye closure and frequent yawning, which can be signs of driver fatigue. 
To prevent incidents caused by drowsiness, our system alerts the driver in advance by analyzing drowsiness indicators.
I am using Python as my programming language and leveraging libraries such as Pandas, NumPy, Dlib, OpenCV, and Math to build and train a driver drowsiness detection model.

Currently, I am working with pretrained data for model training while also developing a custom dataset, which I train using TensorFlow/keras, PyTorch with a Cnn model. To enhance features extraction in dim light conditions since most accidents occur at night, 
I am integrating histogram techniques for improved accuracy in detecting drowsiness indicators.
Additionally, I am working on head movement detection to ensure alertness even when the automobile is in motion and the system fails to detect a face. This feature helps provide timely alerts in situations where traditional face detection methods might not be reliable.

In [51]:
import dlib
dlib.__version__

'19.24.6'

In [53]:
import cv2
cv2.__version__

'4.10.0'

In [55]:
import numpy as np
from math import hypot

In [57]:
# Initialize video capture (Opencv) and dlib's face detector and shape predictor
cap = cv2.VideoCapture(0)
detector = dlib.get_frontal_face_detector()
# loding a pre trained facial landmark predictor model to detect 68 key point on the face
predictor = dlib.shape_predictor("/Users/ashutoshtomar/Desktop/GitHub/heavy_project/Drowsiness_Detection _System/shape_predictor_68_face_landmarks.dat")


# After detecting features, creating a function to calculate midpoint between two points
def mid(p1, p2):
    return int((p1.x + p2.x) / 2), int((p1.y + p2.y) / 2)

In [58]:
# Function to calculate eye aspect ratio (EAR)
def eye_aspect_ratio(eye_landmark, face_landmark):
    left_point = (face_landmark.part(eye_landmark[0]).x, face_landmark.part(eye_landmark[0]).y)
    right_point = (face_landmark.part(eye_landmark[3]).x, face_landmark.part(eye_landmark[3]).y)
    center_top = mid(face_landmark.part(eye_landmark[1]), face_landmark.part(eye_landmark[2]))
    center_bottom = mid(face_landmark.part(eye_landmark[5]), face_landmark.part(eye_landmark[4]))

    hor_line_length = hypot((left_point[0] - right_point[0]), (left_point[1] - right_point[1]))
    ver_line_length = hypot((center_top[0] - center_bottom[0]), (center_top[1] - center_bottom[1]))

    ratio = hor_line_length / ver_line_length
    return ratio

# Function to calculate mouth aspect ratio (MAR)

def mouth_aspect_ratio(lips_landmark, face_landmark):
    left_point = (face_landmark.part(lips_landmark[0]).x, face_landmark.part(lips_landmark[0]).y)
    right_point = (face_landmark.part(lips_landmark[2]).x, face_landmark.part(lips_landmark[2]).y)
    center_top = (face_landmark.part(lips_landmark[1]).x, face_landmark.part(lips_landmark[1]).y)
    center_bottom = (face_landmark.part(lips_landmark[3]).x, face_landmark.part(lips_landmark[3]).y)

    hor_line_length = hypot((left_point[0] - right_point[0]), (left_point[1] - right_point[1]))
    ver_line_length = hypot((center_top[0] - center_bottom[0]), (center_top[1] - center_bottom[1]))
    if hor_line_length == 0:
        return ver_line_length
    ratio = ver_line_length / hor_line_length
    return ratio

# Function to calculate and display histogram for a given ROI
def display_histogram(roi, title):
    roi_gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
    hist = cv2.calcHist([roi_gray], [0], None, [256], [0, 256])
    hist_normalized = cv2.normalize(hist, hist, 0, 255, cv2.NORM_MINMAX)
    hist_image = np.zeros((100, 256), dtype=np.uint8)
    for i in range(256):
        cv2.line(hist_image, (i, 100), (i, 100 - int(hist_normalized[i])), 255, 1)
    cv2.imshow(title, hist_image)

In [59]:
# main loop
count = 0
font = cv2.FONT_HERSHEY_TRIPLEX
while True:
    _, img = cap.read()
    img = cv2.flip(img, 1)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    faces = detector(gray)
    for face in faces:
        landmark_list = predictor(gray, face)

        # (EAR)- eye aspect ratio
        left_eye_ratio = eye_aspect_ratio([36, 37, 38, 39, 40, 41], landmark_list)
        right_eye_ratio = eye_aspect_ratio([42, 43, 44, 45, 46, 47], landmark_list)
        eye_open_ratio = (left_eye_ratio + right_eye_ratio) / 2
        cv2.putText(img, f"EAR: {eye_open_ratio:.2f}", (10, 20), font, 0.5, (100, 100, 100))

        # (MAR) - mouth aspect ratio 
        inner_lip_ratio = mouth_aspect_ratio([60, 62, 64, 66], landmark_list)
        outter_lip_ratio = mouth_aspect_ratio([48, 51, 54, 57], landmark_list)
        mouth_open_ratio = (inner_lip_ratio + outter_lip_ratio) / 2
        cv2.putText(img, f"MAR: {mouth_open_ratio:.2f}", (10, 40), font, 0.5, (100, 100, 100))

        # Drowsiness detection logic
        # If driver is yawming with open eyes it can still detecte it.
        if mouth_open_ratio > 0.30 or eye_open_ratio > 3.90 or eye_open_ratio > 4.0:    # by manualy analysing 
            count += 1
        else:
            count = 0

        # Draw rectangle around the face
        x, y = face.left(), face.top()
        x1, y1 = face.right(), face.bottom()
        if count > 10:
            cv2.rectangle(img, (x, y), (x1, y1), (0, 0, 255), 2)
            cv2.putText(img, "Drowsy", (x, y - 5), font, 0.5, (0, 0, 255))
        else:
            cv2.rectangle(img, (x, y), (x1, y1), (0, 255, 0), 2)

        # Extract eye and mouth regions for histogram analysis
        left_eye_roi = img[landmark_list.part(37).y:landmark_list.part(41).y,landmark_list.part(36).x:landmark_list.part(39).x]
        mouth_roi = img[landmark_list.part(51).y:landmark_list.part(57).y,landmark_list.part(48).x:landmark_list.part(54).x]

        # Display for eye and mouth regions
        if left_eye_roi.size != 0:
            display_histogram(left_eye_roi, "Left Eye")
        if mouth_roi.size != 0:
            display_histogram(mouth_roi, "Mouth")

    # display the output frame for testing
    cv2.imshow("Driver Drowsiness Detection System", img)

    # Exit on 'esc' key press stop the processing model
    key = cv2.waitKey(1)
    if key == 27:
        break

# Release resources
cap.release()
cv2.destroyAllWindows()

/var/folders/7n/sg8y0p8x2jxg2_zcq4f5b4br0000gn/T/ipykernel_3445/472534542.py:36: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  cv2.line(hist_image, (i, 100), (i, 100 - int(hist_normalized[i])), 255, 1)


In [60]:
# for closing detection window
key = cv2.waitKey(1)
if key == 27:  
    break
cap.release()
cv2.destroyAllWindows()

SyntaxError: 'break' outside loop (193630545.py, line 4)